In [4]:
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm # Color maps
from scipy.optimize import minimize

In [2]:
'''
Function definitions for the normal-inverse-gamma distribution. The parameters
of the distribution, namely mu (μ), either lambda (λ) or nu (ν), alpha (α),
beta (β), are used as defined here:

  https://en.wikipedia.org/wiki/Normal-inverse-gamma_distribution

Note that we use the symbol nu (ν) rather than lambda (λ) for the third
parameter. This is to match the notation used in the conjugate priors table on
Wikipedia:

  https://en.wikipedia.org/wiki/Conjugate_prior#Table_of_conjugate_distributions
'''

def norminvgamma_pdf(x, sigma2, mu, nu, alpha, beta):
    '''
    The probability density function of the normal-inverse-gamma distribution at
    x (mean) and sigma2 (variance).
    '''
    return (
        sts.norm.pdf(x, loc=mu, scale=np.sqrt(sigma2 / nu)) *
        sts.invgamma.pdf(sigma2, a=alpha, scale=beta))

def norminvgamma_rvs(mu, nu, alpha, beta, size=1):
    '''
    Generate n samples from the normal-inverse-gamma distribution. This function
    returns a (size x 2) matrix where each row contains a sample, (x, sigma2).
    '''
    # Sample sigma^2 from the inverse-gamma distribution
    sigma2 = sts.invgamma.rvs(a=alpha, scale=beta, size=size)
    # Sample x from the normal distribution
    x = sts.norm.rvs(loc=mu, scale=np.sqrt(sigma2 / nu), size=size)
    return np.vstack((x, sigma2)).transpose()

In [5]:
mu_0 = 0        # The prior mean is centered around 0.
nu_0 = 0.054    # The smaller ν₀ is, the more uncertain we are about the prior mean.
alpha_0 = 1.12  # α₀ and β₀ control the marginal prior over the variance.
beta_0 = 0.4
x_array = np.linspace(-20,20,100)
samples = norminvgamma_rvs(mu_0, nu_0, alpha_0, beta_0, size=10000)

## Method 1: 

In [26]:
def objective_function_1(x):
    mu_0, nu_0, alpha_0, beta_0 = x
    size = 10000
    samples = norminvgamma_rvs(mu_0, nu_0, alpha_0, beta_0, size=size)
    mean_samples = np.array([x[0] for x in samples])
    variance_samples = np.array([x[1] for x in samples])
    mean_constraints = sts.norm.rvs(loc=2.3, scale=0.5, size=size)
    variance_constraints = sts.norm.rvs(loc=2.75, scale=1, size=size)
    return np.log(np.sum((mean_samples-mean_constraints)**2) + np.sum((variance_samples-variance_constraints)**2))

initial_parameters = np.array([0,0.5,1,0.5])
result = minimize(objective_function_1, initial_parameters)

In [27]:
print('mu_0 = ', result.x[0])
print('nu_0 = ', result.x[1])
print('alpha_0 = ', result.x[2])
print('beta_0 = ', result.x[3])

mu_0 =  2.552638946836907e-06
nu_0 =  0.49999704984621884
alpha_0 =  0.9999915505190491
beta_0 =  0.5000102583571149


In [29]:
def objective_function_2(x):
    mu_0, nu_0, alpha_0, beta_0 = x
    size = 10000
    samples = norminvgamma_rvs(mu_0, nu_0, alpha_0, beta_0, size=size)
    mean_samples = np.array([x[0] for x in samples])
    variance_samples = np.array([x[1] for x in samples])
    mean_constraints = sts.norm.rvs(loc=2.3, scale=0.5, size=size)
    variance_constraints = sts.norm.rvs(loc=2.75, scale=1, size=size)
    return np.log((np.mean(mean_samples)-2.3)**2 + (np.std(mean_samples)-0.5)**2 + (np.mean(variance_samples)-2.75)**2 + (np.std(variance_samples)-1)**2)

initial_parameters = np.array([0,0.5,1,0.5])
result = minimize(objective_function_2, initial_parameters)

In [30]:
print('mu_0 = ', result.x[0])
print('nu_0 = ', result.x[1])
print('alpha_0 = ', result.x[2])
print('beta_0 = ', result.x[3])

mu_0 =  4.572657174690074e-05
nu_0 =  0.5000261099501492
alpha_0 =  1.0000048261695238
beta_0 =  0.5000368768900448


## Method 2:

https://drive.google.com/file/d/1e1Sbyjo2CrgiJwo0-EPN3YBmikosYCK2/view